In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
pip install spacy_langdetect

     |████████████████████████████████| 998 kB 2.7 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=a26abc6f2ecb77bacdcf198e39b989a02db140e17658871bd1d69027432eb3ee
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [4]:
pip install pycld2

     |████████████████████████████████| 41.4 MB 1.1 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834142 sha256=3959f1518be8b643d01680ed498ad3f381fa572bd8aa2c6ea3960c6bcde07b71
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built pycld2


In [5]:
import pandas as pd
import re
import re
import string
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
# Individual cleaning functions
def remove_web_links(text):
    text = re.sub(r'http://www.\w+.org/','', text)
    text = re.sub(r'http://www.([\w\S]+).org/\w+\W\w+','',text)
    text = re.sub(r'https://\w+.\w+/\d+.\d+/\w\d+\W\w+','',text)
    text = re.sub(r'https://\w+.\w+/\d+.\d+/\w\d+\W\w+','',text)
    text = re.sub(r'Figure\s\d:','', text)
    text = re.sub(r'\Wwww.\w+\W\w+\W','',text)
    text = re.sub("@[A-Za-z0-9]+", "", text)
    text = re.sub(r'www.\w+','',text)

    return text

def remove_emojis(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    text = regrex_pattern.sub('', text)

    return text

def remove_spaces(text):
    text = re.sub(r'\n',"",text)

    return text


def remove_stopwords(text):
    stop_words=set(stopwords.words('english'))
    words=word_tokenize(text)
    sentence=[w for w in words if w not in stop_words]
    return " ".join(sentence)

def lemmatize_text(text):
    wordlist=[]
    lemmatizer = WordNetLemmatizer()
    sentences=sent_tokenize(text)
    for sentence in sentences:
        words=word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))
    return ' '.join(wordlist)

def lowercase_text(text):
    return text.lower()


def remove_punctuations(text):
    additional_punctuations = ['’', '…'] # punctuations not in string.punctuation  
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
  
    for punctuation in additional_punctuations:
        text = text.replace(punctuation, '')
    
    return text

def remove_numbers(text):
    if text is not None:
        text = text.replace(r'^\d+\.\s+','')
  
    text = re.sub("[0-9]", '', text)
    return text

In [13]:
def clean_and_preprocess_data(text, lowercase=True, clean_stopwords=True, clean_punctuations=True, clean_links=True, 
                              clean_emojis=True, clean_spaces=True, clean_numbers=True,  lemmatize=True):
  
    if clean_stopwords == True:
        text = remove_stopwords(text)

    if clean_punctuations == True:
        text = remove_punctuations(text)
  
    if clean_links == True:
        text = remove_web_links(text)
  
    if clean_emojis == True:
        text = remove_emojis(text)
  
    if clean_spaces == True:
        text = remove_spaces(text)
  
    if clean_numbers == True:
        text = remove_numbers(text)
  
    if lemmatize == True:
        text = lemmatize_text(text)
  
    if lowercase == True:
        return text.lower()

    return text

In [7]:
df2 = pd.read_csv('/content/gdrive/My Drive/Omdena_Normal/full_tag_extracted_results.csv')

In [ ]:
pd.set_option('max_colwidth', None)
df2.head()

In [32]:
#create the dataset for 'UK' and the fashion industry
fashion_data= df2[(df2['industry'] == 'fashion') & (df2['country']=='UK')]
len(fashion_data)

6988

In [33]:
%time 
#remove emojis and extra spaces from the data
fashion_data['paragraph']=fashion_data['paragraph'].apply(lambda x: clean_and_preprocess_data(x,lowercase=False, clean_stopwords=False, clean_punctuations=False, clean_links=False, 
                              clean_emojis=True, clean_spaces=True, clean_numbers=False,  lemmatize=False))

#remove new line and \t from the text data
fashion_data['paragraph']=fashion_data['paragraph'].replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [15]:
fashion_data.head()

,Unnamed: 0,id,resourceLink,tags,paragraph,external,benefit,industry,country
99649,99649,cc559215-afa8-5109-8cda-dffed55feeac.json,http://oro.open.ac.uk/77120/3/77120.pdf,"{'external': ['test'], 'benefit': 'sustainable', 'industry': 'fashion', 'country': 'UK'}","Black, sandy; Williams, Dilys; Rocamora, Agnes; Burcikova, Mila; Lyon, Fergus; Werner, Andrea; Vickers, Ian;Elf, Patrick; Eckert, Claudia and Crommentuijn-Marsh, Philippa (2021). Fostering Sustainable Practices? The case ofmicro and small designer fashion enterprises. In: Sustainable Innovation 2021, 15-21 Mar 2021, Farnborough",['test'],sustainable,fashion,UK
99650,99650,cc559215-afa8-5109-8cda-dffed55feeac.json,http://oro.open.ac.uk/77120/3/77120.pdf,"{'external': ['test'], 'benefit': 'sustainable', 'industry': 'fashion', 'country': 'UK'}","Copyright and Moral Rights for the articles on this site are retained by the individual authors and/or other copyrightowners. For more information on Open Research Online’s data policy on reuse of materials please consult the policiesFostering Sustainable Practices? The case of micro and small Professor of Fashion & Textile Design and Technology Centre for Sustainable Fashion, University of the Arts London, London College of Fashion Director, Centre for Sustainable Fashion and Professor of Fashion Design for Sustainability University of the Arts London, London College of Fashion",['test'],sustainable,fashion,UK
99651,99651,cc559215-afa8-5109-8cda-dffed55feeac.json,http://oro.open.ac.uk/77120/3/77120.pdf,"{'external': ['test'], 'benefit': 'sustainable', 'industry': 'fashion', 'country': 'UK'}","of the UK’s creative industries, and widely recognized for its fashion influence beyond the UK. This paper presents findings from the Rethinking Fashion Design Entrepreneurship: Fostering Sustainable Practices project, working with innovative fashion micro and small enterprises (MSEs) in the UK. With their strong sense of values and purpose, contributing to community alongside maintaining financial viability, these MSEs provide examples of best practice to foster sustainability. Whilst the fashion industry has been severely affected by the Covid-19 pandemic, our investigation revealed some fashion design entrepreneurs showing exceptional resilience, despite facing financial challenges",['test'],sustainable,fashion,UK
99652,99652,cc559215-afa8-5109-8cda-dffed55feeac.json,http://oro.open.ac.uk/77120/3/77120.pdf,"{'external': ['test'], 'benefit': 'sustainable', 'industry': 'fashion', 'country': 'UK'}","Such small and agile enterprises exemplify a redefinition of what fashion business is and can be in future, presenting viable pathways to recovery post-pandemic, and building towards a sustainable The pace of fashion cycles has increased enormously over recent decades, with the advent of fast and faster fashion since the 1990s; this high throughput and high waste model is unsustainable and no longer acceptable in the context of depleted and finite planetary resources, exacerbated by the current climate emergency (Steffen et al 2015). The transition to sustainability is now an imperative Development Goals and accelerated by the 2020 Coronavirus pandemic2. Although pioneers of sustainable fashion have been active since the 1980s, the industry as a whole has been slow to embed sustainability principles, with initiatives in evidence from the early 21st century (Fletcher 2008, 1 At the 2015 Paris Climate Conference (Conference of the Parties COP21) 189 signatories agreed to limit global warming to 1.5°-2° and publish targets for reduction of their greenhouse gas emissions",['test'],sustainable,fashion,UK
99653,99653,cc559215-afa8-5109-8cda-dffed55feeac.json,http://oro.open.ac.uk/77120/3/77120.pdf,"{'external': ['test'], 'benefit': 'sustainable', 'industry': 'fashion', 'country': 'UK'}","2 “The crisis is a catalyst that will shock the industry into change – now is the time to get ready for a post-coronavirus world. … The pandemic will bring values arou

**Language Detection**

First we detect the main language for each paragraph
There are 2 types:

1. Purely other languages -> indexes are stored in other_lang_index

2. Mixed Language - where main language is english -> indexes are stored as keys in low_score_index

In [34]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

low_score_index = {}
other_lang_index = []
nlp = spacy.load("en")

nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
for i in range(0,len(fashion_data)):
    text = fashion_data.iloc[i,4]
    doc = nlp(text)._.language
    if doc['language']=='en':
        if doc['score']<0.99999:
            low_score_index[i] = doc['language']
    elif doc['language']!='en':
        other_lang_index.append(i)
    if i%500==0:
      print(i)
      print('low_score_index',low_score_index)
      print('other_lang_index',other_lang_index)

0
low_score_index {}
other_lang_index []
500
low_score_index {}
other_lang_index []
1000
low_score_index {985: 'en', 986: 'en'}
other_lang_index [825]
1500
low_score_index {985: 'en', 986: 'en'}
other_lang_index [825]
2000
low_score_index {985: 'en', 986: 'en', 1509: 'en', 1849: 'en'}
other_lang_index [825]
2500
low_score_index {985: 'en', 986: 'en', 1509: 'en', 1849: 'en', 2360: 'en'}
other_lang_index [825, 2438, 2439, 2440, 2441, 2442, 2443, 2444]
3000
low_score_index {985: 'en', 986: 'en', 1509: 'en', 1849: 'en', 2360: 'en', 2690: 'en'}
other_lang_index [825, 2438, 2439, 2440, 2441, 2442, 2443, 2444]
3500
low_score_index {985: 'en', 986: 'en', 1509: 'en', 1849: 'en', 2360: 'en', 2690: 'en'}
other_lang_index [825, 2438, 2439, 2440, 2441, 2442, 2443, 2444]
4000
low_score_index {985: 'en', 986: 'en', 1509: 'en', 1849: 'en', 2360: 'en', 2690: 'en'}
other_lang_index [825, 2438, 2439, 2440, 2441, 2442, 2443, 2444]
4500
low_score_index {985: 'en', 986: 'en', 1509: 'en', 1849: 'en', 2360: '

In the second part below, we try to fetch the mixed language paragraphs(from the low score index) and only retain the English sentences in these. 

In [37]:
import pycld2 as cld2

#get the low score indices
low_score_indexes=[i for i in low_score_index.keys()]

#retain the english sentences in the mixed language texts(low score index) and drop the other non-English sentences
for i in low_score_indexes:
  print(i)
  text_content= fashion_data.iloc[i,4]
  _, _, _, detected_language = cld2.detect(text_content,  returnVectors=True)
  eng_sentence_list=[]
  for inner_tuple in detected_language:
      print('Old sentence:', fashion_data.iloc[i,4])
      print(inner_tuple)
      if inner_tuple[3]=='en':
        start_index= inner_tuple[0]
        end_index= inner_tuple[0]+inner_tuple[1]
        eng_sentence_list.append(text_content[start_index:end_index])
        print(eng_sentence_list)
  if eng_sentence_list:
    fashion_data.iloc[i,4]= ' '.join(eng_sentence_list)
    print('New sentence:',fashion_data.iloc[i,4])
  else:
    print('None of the sentences are in English')
    other_lang_index.append(i)
  print('\n')

985
Old sentence: Goldfarb, M. I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J. B. R. Matthews, T. K. Maycock, T. Waterfield, O. Yelekçi, R. Yu and B. Zhou (eds.)]
(0, 144, 'Unknown', 'un')
None of the sentences are in English


986
Old sentence: Goldfarb, M. I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J. B. R. Matthews, T. K. Maycock, T. Waterfield, O. Yelekçi, R. Yu and B. Zhou (eds.)]
(0, 144, 'Unknown', 'un')
None of the sentences are in English


1509
Old sentence: Niemelä, M. (2010). Kestävää muotoilua mallintamassa. Tulkitseva käsitetutkimusInterpretative concept research from the viewpoint of craft design]. doctoraldissertation. Helsinki. Finland: University of Art and Design Helsinki
(0, 94, 'FINNISH', 'fi')
Old sentence: Niemelä, M. (2010). Kestävää muotoilua mallintamassa. Tulkitseva käsitetutkimusInterpretative concept research from the viewpoint of craft design]. doctoraldissertation. Helsinki. Finland: University of Art and Design Helsinki
(94, 137, 'ENGLISH', 'en')
['conce

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [38]:
print(other_lang_index)
#peviously-[825, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 5568]
#now-[825, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 5568, 985, 986, 1849]

[825, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 5568, 985, 986, 1849]
